In [2]:
import pandas as pd
import numpy as np
import xlwt
import xlrd
from tkinter import filedialog
import tkinter as tk
import customtkinter as ctk
from customtkinter import filedialog
import funciones
import openpyxl
import dbfread  


### test cargar db

In [8]:
prueba = funciones.subir_dbf()
print(prueba['IT_ITOT'])


C:/Users/zickd/Downloads/INDICADORESV53.DBF
       ORD_ID ORD_NUMERO  EXP_ID   OS_NOMBRE                      NOMBRE SEXO  \
0       34620  Amb-43725   10132  OSOETSYLRA        INEXISTENTE,AFILIADO    M   
1       34621  Amb-43728   10133  OSOETSYLRA        INEXISTENTE,AFILIADO    M   
2       34946  Amb-44006   10155  OSOETSYLRA         OJEDA,LAUTARO URIEL    M   
3       34947  Amb-44009   10156  OSOETSYLRA         OJEDA,LAUTARO URIEL    M   
4       34948  Amb-44010   10157  OSOETSYLRA         OJEDA,LAUTARO URIEL    M   
...       ...        ...     ...         ...                         ...  ...   
20893   49079      51343   12154  OSOETSYLRA  ESPOSITO,PATRICIA CAROLINA    F   
20894   49080      51342   12154  OSOETSYLRA  ESPOSITO,PATRICIA CAROLINA    F   
20895   49081      51340   12154  OSOETSYLRA  ESPOSITO,PATRICIA CAROLINA    F   
20896   49082  Amb-58807   13020  OSOETSYLRA      AGUIRRE,GENESIS OLIVIA    F   
20897   49083  Amb-58809   13021  OSOETSYLRA      DISCIOSCIA,ALAN

#### leer DBF simpe

In [3]:
class MyFieldParser(dbfread.FieldParser):

    # Sobrescribir el método parseN
    def parseN(self, field, data):
        # Si el campo es "importe", intentar convertir el valor a cadena
        if field.name == "importe":
            try:
                return str(data)
            # Si hay un error, devolver el valor original
            except ValueError:
                return data
        # Si el campo no es "importe", devolver el valor original
        else:
            return data
        
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            dbf =  dbfread.DBF( ruta_archivo, load=True, parserclass=MyFieldParser)
            indicadores = pd.DataFrame(dbf)
            #indicadores = pd.read_excel(ruta_archivo, engine='openpyxl')
            #padron = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")


<_io.TextIOWrapper name='C:/Users/zickd/Downloads/INDICADORESV53.DBF' mode='r' encoding='cp1252'>


In [9]:
prueba['IT_ITOT'].head(30)

0      160437.00
1      110246.00
2      518400.00
3      116000.00
4      255760.00
5        7537.50
6       10752.00
7       93828.50
8      111200.00
9      172510.00
10      14016.00
11       3156.00
12     266000.00
13       2800.00
14      26500.00
15     377520.00
16       7806.73
17       9500.00
18       3500.00
19     136800.00
20      74800.00
21      15200.00
22    2565262.50
23        979.00
24      24500.00
25       2289.00
26       2063.00
27       2574.00
28       2574.00
29       4900.00
Name: IT_ITOT, dtype: float64

### modelado

In [12]:
diccionario = {
'CONSULTA':[420301,
420132,
420125,
420134,
420126,
420127,
420130,
420121,
420117,
420129,
420110,
420104,
420145,
420105,
420102,
420109,
420128,
420108,
420101,
420103,
1,
420120,
3,
420133,
420107,
420112,
420143,
420135,
420106,
101,
420303,
420144,
420141]}

# creo la funcion para iterar el diciccionario

def aplicar_categoria(codigo):
    for categoria, codigos in diccionario.items():
        if codigo in codigos:
            return categoria
    return "practica"


# Crear campo pais
#indicadores['PAIS'] = 'ARGENTINA' + ',' + indicadores['PROV_AFI'] + ',' + indicadores['LOCAAFI']
prueba['PAIS'] = 'ARGENTINA' + ',' + prueba['PROV_AFI'] + ',' + prueba['LOCAAFI']
            

### aplicar diccionario y crear nuevo campo practicas_consultas

In [13]:
#indicadores['CONSULTAS_PRACTICAS']=indicadores['IT_COD'].apply(aplicar_categoria)
prueba['CONSULTAS_PRACTICAS']=prueba['IT_COD'].apply(aplicar_categoria)

#print(indicadores[['IT_COD','CONSULTAS_PRACTICAS']])
#save_path = filedialog.askdirectory()
#test
#indicadores2 = indicadores[[
indicadores2 = prueba[[
"ORD_ID",
"ORD_NUMERO",
"ORD_PERI",
"EXP_ID",
"OS_NOMBRE",
"NOMBRE",
"SEXO",
"EDAD",
"LOCAAFI",
"ORD_FSOL",
"ORD_FEC",
"NOMSOL",
"ORD_ORI",
"ORI_NOM",
"LOCAORI",
"ORD_AREA",
"NOMENCLA",
"IT_COD",
"NOM_NOM",
"CONSULTAS_PRACTICAS",
"CANTIDAD",
"IT_ITOT",
"MOT_NOM",
"NOMGTS",
"TEXP_NOMBR",
"PLAN_NOMBR",
"MONODROGA",
"ACCION",
"PAIS",
"PROV_AFI",
"IT_OBS",
"TIPOORIGEN",
"AGENTE_GTS",
"COB_ESP",
"DOC_ID",
"FECHAALTA"
]]
indicadores2.columns= indicadores2.columns.str.lower()
#indicadores2['it_itot'] = indicadores2['it_itot'].replace('b':'',   regex=True)
#indicadores2.to_excel(save_path + '\indicadoresv5.xlsx', engine='openpyxl' ,index = False )
#indicadores2.to_csv(save_path + '\indicadoresv5.csv' ,index = False )
print(indicadores2['it_itot'])

0         160437.00
1         110246.00
2         518400.00
3         116000.00
4         255760.00
            ...    
20893        559.87
20894       4632.30
20895       4081.50
20896     104000.00
20897    8500019.00
Name: it_itot, Length: 20898, dtype: float64


### SUBIR EL EXCEL CORREGIDO

In [5]:
#cargar primero el excel o csv
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            indicadores3 = pd.read_excel(ruta_archivo)
            #padron = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            print(indicadores)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")


<_io.TextIOWrapper name='C:/Users/zickd/Downloads/indicadoresv5.xlsx' mode='r' encoding='cp1252'>
       ORD_ID ORD_NUMERO  EXP_ID   OS_NOMBRE                  NOMBRE SEXO  \
0       25549  Amb-33444    8032  OSOETSYLRA    INEXISTENTE,AFILIADO    M   
1       25550  Amb-33445    8033  OSOETSYLRA    INEXISTENTE,AFILIADO    M   
2       25639  Amb-33536    8037     VISITAR    GONZALEZ,MARíA BELEN    F   
3       25641  Amb-33562    8052  OSOETSYLRA    PEON,RENZO VALENTINO    M   
4       25642  Amb-33569    8053  OSOETSYLRA        SOSA,AXEL HERNAN    M   
...       ...        ...     ...         ...                     ...  ...   
27148   42333      43593   11950     VISITAR  GIMENEZ,LEANDRO JAVIER    M   
27149   42333      43593   11950     VISITAR  GIMENEZ,LEANDRO JAVIER    M   
27150   42333      43593   11950     VISITAR  GIMENEZ,LEANDRO JAVIER    M   
27151   42333      43593   11950     VISITAR  GIMENEZ,LEANDRO JAVIER    M   
27152   42333      43593   11950     VISITAR  GIMENEZ,L

In [ ]:
indicadores3.style.format({'it_tot': '{:,.2f}'})

In [ ]:
indicadores['IT_ITOT']

### GUARDAR EL DATAFRAME EN UNA HOJA DE GSHEET

In [ ]:
import pygsheets
import pandas as pd

In [1]:
import pygsheets
import pandas as pd

path='C:\\Users\zickd\\OneDrive\Documentos\\repos_git\\signomedico-398902-d596060414cb.json'
gc=pygsheets.authorize(service_account_file=path)
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/10IKRk1MISpvgmRxH1pj_63Kt8PMEEJGvJp8I6unsa3c/edit?usp=drive_link')
wk1=sh[0] # first worksheet 
#wk1.clear() # to remove all data from sheet
wk1.set_dataframe(indicadores2,(1,1), extend=True, copy_head=True)

NameError: name 'indicadores2' is not defined

In [12]:
print(dbfread.__version__)



2.0.7
